## Climatology and Anomaly in Geoscience
An example of how Python is used to explore global sea surface temperature (SST).
* Wenchang Yang (wenchang@princeton.edu)
* Department of Geosciences, Princeton University
* Junior Colloquium, Nov. 08, 2021


## What's covered so far
1. Python basics: `number`, `string`, `list`, `function`, `module`, `package`, ...
2. Scientific computation: `numpy`,
3. Data visualization: `matplotlib`,
4. High-level and user-friendly packages: `pandas` and `xarray`

## Today's plan

1. A short introduction to `xarray`.
2. SST dataset.
3. Scientific questions to keep in mind.
4. Step-by-step data analysis using Python.


## What's `xarray`  able to do? 

<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" width="300">

1. Open/save datasets (single/multiple, local/remote): `open_dataset`, `open_mfdataset`.
2. Data selection: `sel`, `isel`.
3. Computation: `mean`, `std`, `max`, `min`, `differentiate`, `integrate`, ...
4. Split-apply-combine: `groupby`.
5. Plot: `plot`, `plot.line`, `plot.contourf`, `plot.hist` ...

http://xarray.pydata.org/

## SST data
* ERSST version 5: global **monthly** SST.
* $2^\circ$ longitude $\times$ $2^\circ$ latitude 
* From Columbia University [data library](http://iridl.ldeo.columbia.edu): http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/T/%28Jan%201979%29/%28Dec%202018%29/RANGE/X//lon/renameGRID/Y//lat/renameGRID/T//time/renameGRID/time/(days%20since%201979-01-01)/streamgridunitconvert%5Bzlev%5Daverage
* It covers 1854-present, but we focus on **1979-2018** today.
* Downloaded and available on the Adroit server:<br> `/home/wenchang/JC2021/ersst5_1979-2018.nc`

## Some scientific questions to keep in mind
1. What does global SST pattern look like?
2. Is SST getting warmer over the recent decades?
2. How do El Nino/La Nina vary during this period?
    
![](https://blog.weatherops.com/hubfs/blog-files/elnino-vs-lanina-noaa.jpg)
https://blog.weatherops.com/hubfs/blog-files/elnino-vs-lanina-noaa.jpg

## Start analysis

In [ ]:
# xarray is the core package we are going to use
import xarray as xr
import matplotlib.pyplot as plt # we also use pyplot directly in some cases
import os

In [ ]:
# some configurations on the default figure output
%config InlineBackend.figure_format ='retina'
plt.rcParams['figure.dpi'] = 120

## Open the SST data file
Use `xr.open_dataset` to open the SST data file.


In [ ]:
# Am I running the notebook on Adroit or not?
if os.uname().nodename.startswith('adroit'):
    ifile = '/home/wenchang/JC2021/ersst5_1979-2018.nc'
else:
    ifile = 'ersst5_1979-2018.nc'
print('file to be opened:',ifile)
ds = xr.open_dataset(ifile)

In [ ]:
ds

In [ ]:
sst = ds.sst
sst

## Explore the data by making simple plots

In [ ]:
# first time point
sst.isel(time=0).plot()
# sst.plot(bins=100, density=True, cumulative=True)

More experiments on plotting:
* select date/time explicitly
* change colormap/levels
* contourf/contour


In [ ]:
# specify date/time explicitly
# sst.isel(time=0).plot()
sst.sel(time='1979-01').plot()

In [ ]:
# change colormap levels
sst.sel(time='1979-01-16T12').plot(levels=21)

In [ ]:
# change color map to 'Spectral_r'. 
# More colormaps: https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
sst.sel(time='1979-01-16T12').plot(levels=10, center=False, cmap='Spectral_r')

In [ ]:
# change to contourf
sst.sel(time='1979-01-16T12').plot.contourf(levels=10, center=False, cmap='Spectral_r')

## 40-year annual mean climatology

In [ ]:
sst_clim = sst.mean('time')

In [ ]:
sst_clim

In [ ]:
sst_clim.plot.contourf(levels=10, cmap='Spectral_r', center=False)

* Warm tropics and cold polar regions.
* Indo-Pacific Warm Pool.

## SST change from the first 10 years to the last 10 years

In [ ]:
sst_early = sst.sel(time=slice('1979-01', '1988-12')).mean('time')
sst_late = sst.sel(time=slice('2009-01', '2018-12')).mean('time')
dsst = sst_late - sst_early
dsst.attrs['long_name'] = 'SST change from 1979-1988 to 2009-2018'
dsst.attrs['units'] = '$^\circ$C'

In [ ]:
dsst

In [ ]:
dsst.plot.contourf(levels=21)
# cooling over the Southern Ocean and Southeast Pacific

* Not warming everywhere. 
* Cooling over the Southern Ocean and Southeast Pacific

## Calculate monthly climatology
* multiple-year mean for each of the 12 months
* use the `groupby('time.month')` method



In [ ]:
sst_mclim = sst.groupby('time.month').mean('time')
# the time dimension is replaced by month

In [ ]:
sst_mclim

In [ ]:
# Jan vs. Jul
fig, axes = plt.subplots(1, 2, figsize=(9,3))

sst_mclim.sel(month=1).plot.contourf(levels=range(-2,35, 2), cmap='Spectral_r', ax=axes[0])
sst_mclim.sel(month=7).plot.contourf(levels=range(-2,35, 2), cmap='Spectral_r', ax=axes[1])

fig.tight_layout()

Make 12 subplots using a single command.

In [ ]:
sst_mclim.plot.contourf(col='month', col_wrap=6, levels=20, 
                        cmap='Spectral_r', center=False)

## Calculate monthly anomaly
Subtract the monthly climatology from the raw SST data.

In [ ]:
ssta = sst.groupby('time.month') - sst_mclim
# monthly climatology is now removed

In [ ]:
ssta

In [ ]:
# The 1997 winter is a big El Nino season
ssta.sel(time=slice('1997-12', '1998-02')).mean('time').plot.contourf(levels=19)

In [ ]:
# The 1998 winter is a big La Nina season
ssta.sel(time=slice('1998-12', '1999-02')).mean('time').plot.contourf(levels=19)

In [ ]:
# The 2015 winter is also a big El Nino season
ssta.sel(time=slice('2015-12', '2016-02')).mean('time').plot.contourf(levels=19)

## Calculate the Nino3.4 index
* SST anomaly averaged over the Nino3.4 region: 170W-120W, 5S-5N
![ENSO](http://www.bom.gov.au/climate/enso/indices/oceanic-indices-map.gif)
http://www.bom.gov.au/climate/enso/indices/oceanic-indices-map.gif

In [ ]:
nino34 = ssta.sel(lon=slice(360-170,360-120),lat=slice(-5,5)).mean(['lon','lat'])
nino34.attrs['long_name'] = 'Nino3.4 index'

In [ ]:
nino34

In [ ]:
nino34.plot()
plt.axvline('2015-12', color='gray', ls='--')
plt.text('2015-12', 2, '2015-12', rotation=-45, color='gray', )
plt.axvline('1997-12', color='gray', ls='--')
plt.text('1997-12', 2, '1997-12', rotation=-45, color='gray', )
plt.axvline('1982-12', color='gray', ls='--')
plt.text('1982-12', 2, '1982-12', rotation=-45, color='gray', )

## Seasonality of El Nino/La Nina

In [ ]:
# compare Nino3.4 index variability for each month
da = nino34.groupby('time.month').std('time') # standard deviation for each month
da = da.roll(month=-5).assign_coords(month=range(6, 18)) # roll the time series to start from Jun
da.plot()
plt.bar(da.month, da.values, color='lightgray')
plt.ylabel('Nino3.4 standard deviation [$^\circ$C]')
plt.xticks(range(6,18), ['Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May'])
plt.grid(True)

December shows the largest variability!